In [3]:
import boto3
import pandas as pd

# Initialize a session using Amazon DynamoDB
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')  # Specify your region

# Define the table name
table_name = 'NutritionData'

# Create the DynamoDB table if it doesn't exist
try:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': '임신 주수',  # Use the CSV header directly
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': '임신 주수',
                'AttributeType': 'S'  # Assuming '임신 주수' is a string in the CSV
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    table.wait_until_exists()
except dynamodb.meta.client.exceptions.ResourceInUseException:
    # Table already exists
    table = dynamodb.Table(table_name)

# Read the CSV file
csv_file_path = 'updated_pregnancy_nutrition_data.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file_path)

# Iterate over the DataFrame and insert each row into DynamoDB
for index, row in df.iterrows():
    # Convert the row to a dictionary
    item = row.to_dict()

    # Convert numeric values to their appropriate types
    for key in item:
        if pd.notnull(item[key]):  # Check for NaN values
            if isinstance(item[key], float) and item[key].is_integer():
                item[key] = int(item[key])
            elif isinstance(item[key], float):
                item[key] = int(item[key])
            elif isinstance(item[key], int):
                item[key] = int(item[key])
            elif isinstance(item[key], str):
                item[key] = str(item[key])

    # Insert the item into DynamoDB
    table.put_item(Item=item)

print("Data insertion completed.")

Data insertion completed.
